# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name: Nick Elias

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

##  Question 1

1. Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

In [ ]:
# Step 1: Extract html from coffee review article website

import requests

def save_article_html(url, filename):
    # Send the HTTP request to the URL
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Check if request was successful
    if response.status_code == 200:
        # Get the HTML content of the page
        html_content = response.text

        # Save the HTML content to a file
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"HTML content saved to {filename}")
    else:
        print("Failed to retrieve the page.")

# Example: Save HTML content of a coffee maker review article
url = 'https://www.nytimes.com/wirecutter/reviews/best-espresso-machine-grinder-and-accessories-for-beginners/'  # Replace with the actual review article URL
save_article_html(url, 'coffee_maker_review.html')


# Question 2

2. Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

In [ ]:
import re
import spacy
from bs4 import BeautifulSoup
from html import unescape
from spacytextblob.spacytextblob import SpacyTextBlob

# Load the spaCy model and add SpacyTextBlob extension
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

def extract_clean_text_and_analyze_sentiment(html_file):
    # Read the HTML file
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove <script> and <style> elements
    for script_or_style in soup(['script', 'style']):
        script_or_style.decompose()

    # Optionally remove nav and footer sections
    for nav_or_footer in soup.find_all(['nav', 'footer']):
        nav_or_footer.decompose()

    # Extract main article content (if applicable)
    main_content = soup.find('article') or soup.find('div', class_='main-content')
    if main_content:
        article_text = main_content.get_text(separator="\n", strip=True)
    else:
        article_text = soup.get_text(separator="\n", strip=True)

    # Clean up whitespace and HTML entities
    article_text = re.sub(r'\s+', ' ', article_text)  # Normalize spaces
    article_text = unescape(article_text)  # Decode HTML entities

    # Remove specific unwanted phrases like "Advertisement SKIP ADVERTISEMENT"
    article_text = re.sub(r'\bAdvertisement\s*SKIP\s*ADVERTISEMENT\b', '', article_text, flags=re.IGNORECASE)

    # Process the cleaned text with spaCy
    doc = nlp(article_text)

    # Get the polarity score (sentiment analysis) from SpacyTextBlob
    polarity = doc._.blob.polarity  # SpacyTextBlob stores polarity in the ._.polarity attribute

    # Count the number of sentences in the article
    num_sentences = len(list(doc.sents))

    # Print the results
    print(f"Sentiment Polarity: {polarity:.2f} (Positive if >0, Negative if <0, Neutral if =0)")
    print(f"Number of Sentences in the article: {num_sentences}")

    # Optionally, save the cleaned text to a .txt file
    output_file = 'cleaned_coffee_maker_review.txt'  # Desired file name
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(article_text)

    print(f"Cleaned text has been saved to {output_file}")

    # Save the cleaned HTML back to a new HTML file
    cleaned_html_file = 'cleaned_coffee_maker_review.html'  # Desired HTML file name
    with open(cleaned_html_file, 'w', encoding='utf-8') as file:
        file.write(str(soup))  # Write the cleaned soup (HTML) to file
    print(f"Cleaned HTML has been saved to {cleaned_html_file}")

# Example: Analyze sentiment and sentences for the saved article HTML
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
extract_clean_text_and_analyze_sentiment(html_file)

# Question 3

3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

In [10]:
from collections import Counter

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def analyze_frequent_tokens(html_file):
    # Read the HTML file and extract text
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and get the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # Create a list of tokens (converted to lowercase) and filter out stop words, punctuation, and spaces
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_space]

    # Use Counter to count the frequency of each token
    token_frequencies = Counter(tokens)

    # Get the 5 most common tokens
    most_common_tokens = token_frequencies.most_common(5)

    # Print the results
    print("Top 5 Most Frequent Tokens:")
    for token, freq in most_common_tokens:
        print(f"{token}: {freq}")

    print("\nToken Frequencies:")
    for token, freq in token_frequencies.items():
        print(f"{token}: {freq}")

# Example: Analyze frequent tokens for the saved article HTML
html_file = 'cleaned_coffee_maker_review.html'  # Replace with your saved HTML file path
analyze_frequent_tokens(html_file)


Top 5 Most Frequent Tokens:
espresso: 106
machine: 94
breville: 55
shots: 54
barista: 48

Token Frequencies:
4: 1
best: 19
home: 17
espresso: 106
machines: 43
2024: 5
|: 1
reviews: 2
wirecutter: 31
advertisement: 18
skip: 10
content: 1
help: 8
decide: 1
research: 2
log: 1
account: 1
independently: 1
review: 3
recommend: 11
buy: 9
links: 1
earn: 1
commission: 1
learn: 6
deal: 7
breville: 55
barista: 48
touch: 45
machine: 94
$: 40
693: 5
27: 2
kitchen: 7
coffee: 48
updated: 3
november: 2
25: 1
save: 5
connie: 19
park: 19
nyt: 23
justin: 3
vassallo: 2
ciara: 4
murray: 4
jordan: 4
fyi: 1
new: 8
round: 3
testing: 10
profitec: 26
pick: 26
oracle: 29
jet: 29
upgrade: 9
gaggia: 25
classic: 21
evo: 11
pro: 21
making: 21
café: 11
quality: 13
drinks: 14
longer: 5
reserved: 1
hobbyists: 2
baristas: 5
easier: 2
beginner: 3
started: 1
little: 14
practice: 8
lattes: 5
cortados: 3
rival: 1
upscale: 1
changing: 2
pajamas: 1
dozens: 3
think: 10
option: 6
skilled: 3
enthusiasts: 1
alike: 2
powerful: 14
e

# Question 4

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

In [9]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def analyze_frequent_lemmas(html_file):
    # Read the HTML file and extract text
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and get the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # Create a list of lemmatized tokens (converted to lowercase) and filter out stop words and punctuation
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_space]

    # Use Counter to count the frequency of each lemma
    lemma_frequencies = Counter(lemmas)

    # Get the 5 most common lemmas
    most_common_lemmas = lemma_frequencies.most_common(5)

    # Print the results
    print("Top 5 Most Frequent Lemmas:")
    for lemma, freq in most_common_lemmas:
        print(f"{lemma}: {freq}")

    print("\nLemma Frequencies:")
    for lemma, freq in lemma_frequencies.items():
        print(f"{lemma}: {freq}")

# Example: Analyze frequent lemmas for the saved article HTML
html_file = 'cleaned_coffee_maker_review.html'  # Replace with your saved HTML file path
analyze_frequent_lemmas(html_file)

Top 5 Most Frequent Lemmas:
machine: 136
espresso: 107
shot: 98
breville: 55
barista: 53

Lemma Frequencies:
4: 1
best: 7
home: 17
espresso: 107
machines: 1
2024: 5
|: 1
reviews: 1
wirecutter: 31
advertisement: 18
skip: 10
content: 1
help: 11
decide: 1
research: 2
log: 1
account: 1
independently: 1
review: 4
recommend: 13
buy: 11
link: 1
earn: 1
commission: 1
learn: 8
deal: 7
breville: 55
barista: 53
touch: 46
machine: 136
$: 40
693: 5
27: 2
kitchen: 7
coffee: 49
update: 3
november: 2
25: 1
save: 6
connie: 19
park: 19
nyt: 23
justin: 3
vassallo: 2
ciara: 4
murray: 4
jordan: 4
fyi: 1
new: 8
round: 3
testing: 7
profitec: 26
pick: 43
oracle: 29
jet: 29
upgrade: 9
gaggia: 25
classic: 21
evo: 11
pro: 24
make: 27
café: 11
quality: 13
drink: 23
long: 9
reserve: 1
hobbyist: 2
easy: 13
beginner: 15
start: 9
little: 14
practice: 8
latte: 11
cortado: 3
rival: 1
upscale: 1
change: 5
pajama: 1
test: 16
dozen: 3
think: 10
good: 26
option: 10
skilled: 3
enthusiast: 1
alike: 2
powerful: 14
use: 22
cap

# Question 5

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

In [ ]:
import matplotlib.pyplot as plt

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Add SpacyTextBlob to the pipeline
nlp.add_pipe('spacytextblob')

def analyze_sentiment_per_sentence(html_file):
    # Read the HTML file and extract text
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and get the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # List to store sentiment scores of sentences
    sentence_scores_tokens = []

    # Loop over sentences
    for sent in doc.sents:
        # Get the sentiment polarity score for the sentence
        polarity = sent._.blob.sentiment.polarity
        
        # Add the score to the list of sentence scores
        sentence_scores_tokens.append(polarity)

    # Plot histogram of sentiment scores
    plt.figure(figsize=(10, 6))
    plt.hist(sentence_scores_tokens, bins=20, edgecolor='black')
    plt.title("Histogram of Sentence Sentiment Scores")
    plt.xlabel("Sentiment Score")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

    # Return the sentence scores
    return sentence_scores_tokens

# Example: Analyze sentiment scores for the saved article HTML
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
sentence_scores_tokens = analyze_sentiment_per_sentence(html_file)

# Check the most common range of sentiment scores (based on the histogram)
# Most common range of sentiment scores seems to be between -0.2 and 0.2


# Question 6

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Add SpacyTextBlob to the pipeline
nlp.add_pipe('spacytextblob')

def analyze_sentiment_using_lemmas(html_file):
    # Read the HTML file and extract text
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and get the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # List to store sentiment scores of sentences using lemmas
    sentence_scores_lemmas = []

    # Loop over sentences
    for sent in doc.sents:
        # Extract lemmas of the sentence
        lemmas = [token.lemma_ for token in sent if not token.is_stop and not token.is_punct]
        
        # Calculate the sentiment score based on the polarity of each lemma
        if lemmas:
            # Join lemmas into a sentence string
            sentence_text = ' '.join(lemmas)
            # Using the sentiment from SpacyTextBlob
            sentiment = sent._.blob.sentiment.polarity
            sentence_scores_lemmas.append(sentiment)

    # Plot histogram of sentiment scores based on lemmas
    plt.figure(figsize=(10, 6))
    plt.hist(sentence_scores_lemmas, bins=20, edgecolor='black')
    plt.title("Histogram of Sentence Sentiment Scores (Using Lemmas)")
    plt.xlabel("Sentiment Score")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

    # Return the sentence scores
    return sentence_scores_lemmas

# Example: Analyze sentiment scores for the saved article HTML
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
sentence_scores_lemmas = analyze_sentiment_using_lemmas(html_file)

# Check the most common range of sentiment scores (based on the histogram)
# Most common range of sentiment scores seems to be between -0.2 and 0.2

# Question 7

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

In [ ]:
import numpy as np

def find_cutoff(scores):
    # Sort the scores in ascending order
    sorted_scores = sorted(scores)
    
    # Find the index that corresponds to the point where fewer than half the sentences have a higher score
    cutoff_index = len(sorted_scores) // 2  # Median index (just below the middle)
    
    # The cutoff score will be the score at that index
    cutoff_score = sorted_scores[cutoff_index]
    
    return cutoff_score

# Example: Calculate cutoff for sentiment scores of tokens and lemmas
cutoff_tokens = find_cutoff(sentence_scores_tokens)  # Replace sentence_scores_tokens with the actual scores from tokens
cutoff_lemmas = find_cutoff(sentence_scores_lemmas)  # Replace sentence_scores_lemmas with the actual scores from lemmas

print(f"Cutoff Score (tokens): {cutoff_tokens}")
print(f"Cutoff Score (lemmas): {cutoff_lemmas}")


# Question 8

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

In [ ]:
def create_summary_based_on_tokens(html_file, cutoff_score_tokens):
    # Read and parse the HTML as before
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and extract the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # List to store sentences for the summary
    sentence_list = []

    # Loop over sentences and check their sentiment score (based on tokens)
    for sent in doc.sents:
        # Get the sentiment score of the sentence (based on tokens)
        sentiment = sent._.blob.sentiment.polarity
        
        # If the sentiment score is greater than the cutoff, add the sentence to the summary
        if sentiment > cutoff_score_tokens:
            sentence_list.append(sent.text.strip())  # Add the sentence to the list

    # Join the sentences together to create the summary
    summary = ' '.join(sentence_list)
    
    # Print the summary
    print("Summary:")
    print(summary)

# Example: Generate summary for the saved article HTML
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
create_summary_based_on_tokens(html_file, cutoff_tokens)


# Question 9

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

In [ ]:
def generate_summary_and_analyze_sentiment(html_file, cutoff_score_tokens):
    # Read and parse the HTML as before
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and extract the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # List to store sentences for the summary
    sentence_list = []

    # Loop over sentences and check their sentiment score (based on tokens)
    for sent in doc.sents:
        # Get the sentiment score of the sentence (based on tokens)
        sentiment = sent._.blob.sentiment.polarity
        
        # If the sentiment score is greater than the cutoff, add the sentence to the summary
        if sentiment > cutoff_score_tokens:
            sentence_list.append(sent.text.strip())  # Add the sentence to the list

    # Join the sentences together to create the summary
    summary = ' '.join(sentence_list)
    
    # Print the summary
    print("Summary:")
    print(summary)

    # Calculate the polarity score of the summary
    summary_doc = nlp(summary)  # Process the summary with spaCy
    summary_polarity = summary_doc._.sentiment.polarity  # Get polarity of the summary

    # Count the number of sentences in the summary
    num_sentences_in_summary = len(sentence_list)

    # Print the polarity and number of sentences
    print(f"\nPolarity Score of the Summary: {summary_polarity}")
    print(f"Number of Sentences in the Summary: {num_sentences_in_summary}")

# Example: Generate summary and print its polarity and number of sentences
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
generate_summary_and_analyze_sentiment(html_file, cutoff_tokens)


# Question 10:

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

In [ ]:
def generate_summary_based_on_lemmas(html_file, cutoff_score_lemmas):
    # Read and parse the HTML as before
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and extract the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # List to store sentences for the summary
    sentence_list = []

    # Loop over sentences and check their sentiment score (based on lemmas)
    for sent in doc.sents:
        # Get the sentiment score of the sentence (based on lemmas)
        sentiment = sent._.blob.sentiment.polarity
        
        # If the sentiment score is greater than the cutoff, add the sentence to the summary
        if sentiment > cutoff_score_lemmas:
            sentence_list.append(sent.text.strip())  # Add the sentence to the list

    # Join the sentences together to create the summary
    summary = ' '.join(sentence_list)
    
    # Print the summary
    print("Summary:")
    print(summary)

# Example: Generate summary for the saved article HTML
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
generate_summary_based_on_lemmas(html_file, cutoff_lemmas)


# Question 11:

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

In [ ]:
def generate_summary_and_analyze_sentiment_lemmas(html_file, cutoff_score_lemmas):
    # Read and parse the HTML as before
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    # Parse HTML and extract the text
    soup = BeautifulSoup(html_content, 'html.parser')
    article_text = soup.get_text(separator="\n", strip=True)

    # Process the text with spaCy
    doc = nlp(article_text)

    # List to store sentences for the summary
    sentence_list = []

    # Loop over sentences and check their sentiment score (based on lemmas)
    for sent in doc.sents:
        # Get the sentiment score of the sentence (based on lemmas)
        sentiment = sent._.blob.sentiment.polarity
        
        # If the sentiment score is greater than the cutoff, add the sentence to the summary
        if sentiment > cutoff_score_lemmas:
            sentence_list.append(sent.text.strip())  # Add the sentence to the list

    # Join the sentences together to create the summary
    summary = ' '.join(sentence_list)
    
    # Print the summary
    print("Summary:")
    print(summary)

    # Calculate the polarity score of the summary
    summary_doc = nlp(summary)  # Process the summary with spaCy
    summary_polarity = summary_doc._.blob.sentiment.polarity  # Get polarity of the summary

    # Count the number of sentences in the summary
    num_sentences_in_summary = len(sentence_list)

    # Print the polarity and number of sentences
    print(f"\nPolarity Score of the Lemma-Based Summary: {summary_polarity}")
    print(f"Number of Sentences in the Lemma-Based Summary: {num_sentences_in_summary}")

# Example: Generate lemma-based summary and print its polarity and number of sentences
html_file = 'coffee_maker_review.html'  # Replace with your saved HTML file path
generate_summary_and_analyze_sentiment_lemmas(html_file, cutoff_lemmas)


# Question 12

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

# Question 13

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?